In [29]:
import requests
api_endpoint = 'https://nubela.co/proxycurl/api/v2/linkedin'
linkedin_profile_url = 'https://www.linkedin.com/in/miguel-vela/'
api_key = 'tzCGmD0gDTvHPqqq_XWdpg	'
headers = {'Authorization': 'Bearer ' + api_key}

response = requests.get(api_endpoint,
                        params={'url': linkedin_profile_url,'skills': 'include'},
                        headers=headers)

profile_data = response.json()
profile_data

{'public_identifier': 'miguel-vela',
 'profile_pic_url': 'https://media.licdn.com/dms/image/D4D03AQHcVFBeuLVWKA/profile-displayphoto-shrink_800_800/0/1718218947419?e=1724889600&v=beta&t=_Ihxw2Y_ikpbVGtLtbhR9qKKiSMT2XhE52RZ0FL42Tk',
 'background_cover_image_url': 'https://media.licdn.com/dms/image/D4D16AQEw6esfyrjC_g/profile-displaybackgroundimage-shrink_350_1400/0/1703060277502?e=1724889600&v=beta&t=2K9ocIDhFPqebazZv9tNZpbDgC7diqz0x904eKIMDwg',
 'first_name': 'Miguel',
 'last_name': 'Vela Planas',
 'full_name': 'Miguel Vela Planas',
 'follower_count': 484,
 'occupation': 'Teacher Assistant at The Bridge | Data Scientist | Data Analyst | PowerBI | SQL | MongoDB | AWS | GCP | Pyspark',
 'headline': 'Teacher Assistant at The Bridge | Data Scientist | Data Analyst | PowerBI | SQL | MongoDB | AWS | GCP | Pyspark',
 'summary': 'Como Data Scientist destaco por mi capacidad de adaptación y resiliencia ante los desafíos. Poseo sólidas habilidades de trabajo en equipo, las cuales he demostrado e

In [30]:
print(profile_data['full_name'])
profile_data["profile_pic_url"]

Miguel Vela Planas


'https://media.licdn.com/dms/image/D4D03AQHcVFBeuLVWKA/profile-displayphoto-shrink_800_800/0/1718218947419?e=1724889600&v=beta&t=_Ihxw2Y_ikpbVGtLtbhR9qKKiSMT2XhE52RZ0FL42Tk'

In [31]:
print(profile_data['occupation'])
print(profile_data['summary'])
print(profile_data['skills'])

Teacher Assistant at The Bridge | Data Scientist | Data Analyst | PowerBI | SQL | MongoDB | AWS | GCP | Pyspark
Como Data Scientist destaco por mi capacidad de adaptación y resiliencia ante los desafíos. Poseo sólidas habilidades de trabajo en equipo, las cuales he demostrado en proyectos colaborativos durante mi formación, así como en entornos laborales anteriores.

Mi habilidad para comunicar de manera clara y efectiva, junto con mi don de gentes y disposición para colaborar, me permiten interactuar de manera fluida con compañeros de equipo y clientes. Mi capacidad de resolución de problemas y gestión efectiva de proyectos son el reflejo de estas cualidades.

Destaco por mi flexibilidad y responsabilidad en el trabajo, lo que me permite adaptarme a diferentes contextos y asumir nuevas responsabilidades con determinación. Mi experiencia incluye el dominio de herramientas como Python, SQL, y Git, así como técnicas de Machine Learning, visualización de datos, y Data Cleaning.

Además, t

In [32]:
import openai
import os
os.environ["OPENAI_API_KEY"]= "sk-proj-"
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-proj-",
)

good_review = "El servicio fue excelente y me atendieron muy bien. Muy recomendable."
bad_review = "El servicio fue terrible y me hicieron esperar mucho tiempo. No lo recomiendo."

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

good_embedding = get_embedding(good_review)
bad_embedding = get_embedding(bad_review)

import numpy as np
from numpy.linalg import norm

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

good_reviews = []
bad_reviews = []

for index, row in reviews_rating_2.iterrows():
    review_text = row['review_text']
    review_embedding = get_embedding(review_text)
    similarity_to_good = cosine_similarity(review_embedding, good_embedding)
    similarity_to_bad = cosine_similarity(review_embedding, bad_embedding)
    
    if similarity_to_good > similarity_to_bad:
        good_reviews.append(review_text)
    else:
        bad_reviews.append(review_text)

ModuleNotFoundError: No module named 'openai'

In [ ]:
import openai
import os
import numpy as np
from numpy.linalg import norm

# Configuración de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model).data[0].embedding

# Función para calcular la similitud coseno
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (norm(v1) * norm(v2))


In [ ]:
def create_candidate_embedding(profile):
    work_embedding = get_embedding(profile['work_experience'])
    skills_embedding = get_embedding(profile['skills'])
    about_embedding = get_embedding(profile['about'])
    
    # Combinar las embeddings de las diferentes secciones
    combined_embedding = np.mean([work_embedding, skills_embedding, about_embedding], axis=0)
    return combined_embedding

def match_candidates(candidate_profiles):
    embeddings = [create_candidate_embedding(profile) for profile in candidate_profiles]
    
    matches = []
    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            similarity = cosine_similarity(embeddings[i], embeddings[j])
            matches.append((candidate_profiles[i]['name'], candidate_profiles[j]['name'], similarity))
    
    # Ordenar por similitud en orden descendente
    matches.sort(key=lambda x: x[2], reverse=True)
    return matches

# Ejemplo de uso
candidate_profiles = [
    scrape_linkedin_profile("https://www.linkedin.com/in/example1"),
    scrape_linkedin_profile("https://www.linkedin.com/in/example2"),
    # Agregar más perfiles según sea necesario
]

matches = match_candidates(candidate_profiles)
for match in matches:
    print(f"Match entre {match[0]} y {match[1]} con similitud: {match[2]:.2f}")
